In [1]:
import networkx as nx
import numpy as np
import matplotlib.pyplot as plt
import random
import numpy as np
from mpl_toolkits import mplot3d
from multiprocessing import Pool, TimeoutError

In [2]:
n = 50
def g_gen(q):
    global n
    g = nx.grid_2d_graph(n,n)
    gnp = nx.gnp_random_graph(n**2,q)
    for i,j in gnp.edges:
        g.add_edge(i,j)
    return g

In [3]:
def infect(p, g):
    global n
    infectious = [random.choice(list(g.nodes()))]
    time = 1
    infected = []
    while infectious:
        infected += infectious #add infectious nodes to infected list
        time += 1
        infectious_next = []
        for v in infectious:
            for u in g.neighbors(v):
                if random.random() <= p: #infect with prob p
                    infectious_next.append(u)
        infectious = [v for v in set(infectious_next) if v not in infected]
    return time, len(infected)/n

In [4]:
def get_avg(p,m,q):
    times = []
    infected = []
    for i in range(m):
        t, i = infect(p, g_gen(q))
        times.append(t)
        infected.append(i)
    return sum(times)/m, sum(infected)/m

In [ ]:
times = []
infected = []
qplot = []
pplot = []

m = 10
qlist= list(np.logspace(-4,0,10))

ptests = list(np.linspace(0,0.5,50)) 
ptests += list(np.linspace(0.5,0.8,120)) 
ptests += list(np.linspace(0.8,1,20))

def run_tests(q):
    global ptests
    times = []
    infected = []
    qplot = []
    pplot = []
    for p in ptests:
        qplot.append(q)
        pplot.append(p)
        t, i = get_avg(p,m,q)
        times.append(t)
        infected.append(i)
    return times, infected, qplot, pplot

if __name__ == '__main__':
    # start 4 worker processes
    with Pool(processes=4) as pool:
        # list of [[times, infected, qplot, pplot for certain q], ...]
        l = list(pool.map(run_tests, qlist))
    for i in range(len(l)):
        cur = l[i]            
        times += cur[0]
        infected += cur[1]
        qplot += cur[2]
        pplot += cur[3]


In [ ]:
d = {'ps': pPlot, 'q': qplot, 'size': infected, 'times': times}
df = pd.DataFrame(data=d)
df.to_csv(index=False)

In [ ]:
ax = plt.axes(projection='3d')
ax.plot_trisurf(qplot,ptests,infected, cmap='viridis', edgecolor='none')
plt.ylabel("Transmission probability, p")
plt.xlabel("Long range probability, q")

In [ ]:
ax = plt.axes(projection='3d')
ax.plot_trisurf(qplot,ptests,times, cmap='viridis', edgecolor='none')
plt.ylabel("Transmission probability, p")
plt.xlabel("Long range probability, q")

In [ ]:
plt.scatter(ptests,infected)